# Import libraries

## Imported libraries for DDQN 

In [ ]:
''' 
    Before run into source code, we recommend using provided Docker image  
    Provided and document: TODO: INSERT LINK TO THE DOCUMENT HERE 
''' 
# import os
# Import TF and Keras 
# from keras.layers import Dense, Activation 
# from keras.models import Sequential, load_model, model_from_json
from keras.optimizers import Adam 
# from keras import backend as K
# import tensorflow as tf
from random import random, randrange
# import numpy as np

## Default imported libraries for RLComp2020

In [ ]:
import sys
import numpy as np
import pandas as pd
import datetime
import json
from array import *
import os
import math
# from random import randrange
# import random

from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Activation
from keras import optimizers

import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras import backend as K
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# GAME_SOCKET_DUMMY 

## ObstacleInfo

In [ ]:
#Classes in GAME_SOCKET_DUMMY.py
class ObstacleInfo:
    # initial energy for obstacles: Land (key = 0): -1, Forest(key = -1): 0 (random), Trap(key = -2): -10, Swamp (key = -3): -5
    types = {0: -1, -1: 0, -2: -10, -3: -5}

    def __init__(self):
        self.type = 0
        self.posx = 0
        self.posy = 0
        self.value = 0
        
class GoldInfo:
    def __init__(self):
        self.posx = 0
        self.posy = 0
        self.amount = 0

    def loads(self, data):
        golds = []
        for gd in data:
            g = GoldInfo()
            g.posx = gd["posx"]
            g.posy = gd["posy"]
            g.amount = gd["amount"]
            golds.append(g)
        return golds

class PlayerInfo:
    STATUS_PLAYING = 0
    STATUS_ELIMINATED_WENT_OUT_MAP = 1
    STATUS_ELIMINATED_OUT_OF_ENERGY = 2
    STATUS_ELIMINATED_INVALID_ACTION = 3
    STATUS_STOP_EMPTY_GOLD = 4
    STATUS_STOP_END_STEP = 5

    def __init__(self, id):
        self.playerId = id
        self.score = 0
        self.energy = 0
        self.posx = 0
        self.posy = 0
        self.lastAction = -1
        self.status = PlayerInfo.STATUS_PLAYING
        self.freeCount = 0

class GameInfo:
    def __init__(self):
        self.numberOfPlayers = 1
        self.width = 0
        self.height = 0
        self.steps = 100
        self.golds = []
        self.obstacles = []

    def loads(self, data):
        m = GameInfo()
        m.width = data["width"]
        m.height = data["height"]
        m.golds = GoldInfo().loads(data["golds"])
        m.obstacles = data["obstacles"]
        m.numberOfPlayers = data["numberOfPlayers"]
        m.steps = data["steps"]
        return m

class UserMatch:
    def __init__(self):
        self.playerId = 1
        self.posx = 0
        self.posy = 0
        self.energy = 50
        self.gameinfo = GameInfo()

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

class StepState:
    def __init__(self):
        self.players = []
        self.golds = []
        self.changedObstacles = []

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

## GameSocket

In [ ]:
#Main class in GAME_SOCKET_DUMMY.py
class GameSocket:
    bog_energy_chain = {-5: -20, -20: -40, -40: -100, -100: -100}

    def __init__(self):
        self.stepCount = 0
        self.maxStep = 0
        self.mapdir = "Maps"  # where to load all pre-defined maps
        self.mapid = ""
        self.userMatch = UserMatch()
        self.user = PlayerInfo(1)
        self.stepState = StepState()
        self.maps = {}  # key: map file name, value: file content
        self.map = []  # running map info: 0->Land, -1->Forest, -2->Trap, -3:Swamp, >0:Gold
        self.energyOnMap = []  # self.energyOnMap[x][y]: <0, amount of energy which player will consume if it move into (x,y)
        self.E = 50
        self.resetFlag = True
        self.craftUsers = []  # players that craft at current step - for calculating amount of gold
        self.bots = []
        self.craftMap = {}  # cells that players craft at current step, key: x_y, value: number of players that craft at (x,y)

    def init_bots(self):
        self.bots = [Bot1(2), Bot2(3), Bot3(4)]  # use bot1(id=2), bot2(id=3), bot3(id=4)
        for (bot) in self.bots:  # at the beginning, all bots will have same position, energy as player
            bot.info.posx = self.user.posx
            bot.info.posy = self.user.posy
            bot.info.energy = self.user.energy
            bot.info.lastAction = -1
            bot.info.status = PlayerInfo.STATUS_PLAYING
            bot.info.score = 0
            self.stepState.players.append(bot.info)
        self.userMatch.gameinfo.numberOfPlayers = len(self.stepState.players)
        print("numberOfPlayers: ", self.userMatch.gameinfo.numberOfPlayers)

    def reset(self, requests):  # load new game by given request: [map id (filename), posx, posy, initial energy]
        # load new map
        self.reset_map(requests[0])
        self.userMatch.posx = int(requests[1])
        self.userMatch.posy = int(requests[2])
        self.userMatch.energy = int(requests[3])
        self.userMatch.gameinfo.steps = int(requests[4])
        self.maxStep = self.userMatch.gameinfo.steps

        # init data for players
        self.user.posx = self.userMatch.posx  # in
        self.user.posy = self.userMatch.posy
        self.user.energy = self.userMatch.energy
        self.user.status = PlayerInfo.STATUS_PLAYING
        self.user.score = 0
        self.stepState.players = [self.user]
        self.E = self.userMatch.energy
        self.resetFlag = True
        self.init_bots()
        self.stepCount = 0

    def reset_map(self, id):  # load map info
        self.mapId = id
        self.map = json.loads(self.maps[self.mapId])
        self.userMatch = self.map_info(self.map)
        self.stepState.golds = self.userMatch.gameinfo.golds
        self.map = json.loads(self.maps[self.mapId])
        self.energyOnMap = json.loads(self.maps[self.mapId])
        for x in range(len(self.map)):
            for y in range(len(self.map[x])):
                if self.map[x][y] > 0:  # gold
                    self.energyOnMap[x][y] = -4
                else:  # obstacles
                    self.energyOnMap[x][y] = ObstacleInfo.types[self.map[x][y]]

    def connect(self): # simulate player's connect request
        print("Connected to server.")
        for mapid in range(len(Maps)):
            filename = "map" + str(mapid)
            print("Found: " + filename)
            self.maps[filename] = str(Maps[mapid])

    def map_info(self, map):  # get map info
        # print(map)
        userMatch = UserMatch()
        userMatch.gameinfo.height = len(map)
        userMatch.gameinfo.width = len(map[0])
        i = 0
        while i < len(map):
            j = 0
            while j < len(map[i]):
                if map[i][j] > 0:  # gold
                    g = GoldInfo()
                    g.posx = j
                    g.posy = i
                    g.amount = map[i][j]
                    userMatch.gameinfo.golds.append(g)
                else:  # obstacles
                    o = ObstacleInfo()
                    o.posx = j
                    o.posy = i
                    o.type = -map[i][j]
                    o.value = ObstacleInfo.types[map[i][j]]
                    userMatch.gameinfo.obstacles.append(o)
                j += 1
            i += 1
        return userMatch

    def receive(self):  # send data to player (simulate player's receive request)
        if self.resetFlag:  # for the first time -> send game info
            self.resetFlag = False
            data = self.userMatch.to_json()
            for (bot) in self.bots:
                bot.new_game(data)
            # print(data)
            return data
        else:  # send step state
            self.stepCount = self.stepCount + 1
            if self.stepCount >= self.maxStep:
                for player in self.stepState.players:
                    player.status = PlayerInfo.STATUS_STOP_END_STEP
            data = self.stepState.to_json()
            for (bot) in self.bots:  # update bots' state
                bot.new_state(data)
            # print(data)
            return data

    def send(self, message):  # receive message from player (simulate send request from player)
        if message.isnumeric():  # player send action
            self.resetFlag = False
            self.stepState.changedObstacles = []
            action = int(message)
            # print("Action = ", action)
            self.user.lastAction = action
            self.craftUsers = []
            self.step_action(self.user, action)
            for bot in self.bots:
                if bot.info.status == PlayerInfo.STATUS_PLAYING:
                    action = bot.next_action()
                    bot.info.lastAction = action
                    # print("Bot Action: ", action)
                    self.step_action(bot.info, action)
            self.action_5_craft()
            for c in self.stepState.changedObstacles:
                self.map[c["posy"]][c["posx"]] = -c["type"]
                self.energyOnMap[c["posy"]][c["posx"]] = c["value"]

        else:  # reset game
            requests = message.split(",")
            print("Reset game: ", requests)
            self.reset(requests)

    def step_action(self, user, action):
        switcher = {
            0: self.action_0_left,
            1: self.action_1_right,
            2: self.action_2_up,
            3: self.action_3_down,
            4: self.action_4_free,
            5: self.action_5_craft_pre
        }
        func = switcher.get(action, self.invalidAction)
        func(user)

    def action_5_craft_pre(self, user):  # collect players who craft at current step
        user.freeCount = 0
        if self.map[user.posy][user.posx] <= 0:  # craft at the non-gold cell
            user.energy -= 10
            if user.energy <= 0:
                user.status = PlayerInfo.STATUS_ELIMINATED_OUT_OF_ENERGY
                user.lastAction = 6 #eliminated
        else:
            user.energy -= 5
            if user.energy > 0:
                self.craftUsers.append(user)
                key = str(user.posx) + "_" + str(user.posy)
                if key in self.craftMap:
                    count = self.craftMap[key]
                    self.craftMap[key] = count + 1
                else:
                    self.craftMap[key] = 1
            else:
                user.status = PlayerInfo.STATUS_ELIMINATED_OUT_OF_ENERGY
                user.lastAction = 6 #eliminated

    def action_0_left(self, user):  # user go left
        user.freeCount = 0
        user.posx = user.posx - 1
        if user.posx < 0:
            user.status = PlayerInfo.STATUS_ELIMINATED_WENT_OUT_MAP
            user.lastAction = 6 #eliminated
        else:
            self.go_to_pos(user)

    def action_1_right(self, user):  # user go right
        user.freeCount = 0
        user.posx = user.posx + 1
        if user.posx >= self.userMatch.gameinfo.width:
            user.status = PlayerInfo.STATUS_ELIMINATED_WENT_OUT_MAP
            user.lastAction = 6 #eliminated
        else:
            self.go_to_pos(user)

    def action_2_up(self, user):  # user go up
        user.freeCount = 0
        user.posy = user.posy - 1
        if user.posy < 0:
            user.status = PlayerInfo.STATUS_ELIMINATED_WENT_OUT_MAP
            user.lastAction = 6 #eliminated
        else:
            self.go_to_pos(user)

    def action_3_down(self, user):  # user go right
        user.freeCount = 0
        user.posy = user.posy + 1
        if user.posy >= self.userMatch.gameinfo.height:
            user.status = PlayerInfo.STATUS_ELIMINATED_WENT_OUT_MAP
            user.lastAction = 6 #eliminated
        else:
            self.go_to_pos(user)

    def action_4_free(self, user):  # user free
        user.freeCount += 1
        if user.freeCount == 1:
            user.energy += int(self.E / 4)
        elif user.freeCount == 2:
            user.energy += int(self.E / 3)
        elif user.freeCount == 3:
            user.energy += int(self.E / 2)
        else:
            user.energy = self.E
        if user.energy > self.E:
            user.energy = self.E

    def action_5_craft(self):
        craftCount = len(self.craftUsers)
        # print ("craftCount",craftCount)
        if (craftCount > 0):
            for user in self.craftUsers:
                x = user.posx
                y = user.posy
                key = str(user.posx) + "_" + str(user.posy)
                c = self.craftMap[key]
                m = min(math.ceil(self.map[y][x] / c), 50)
                user.score += m
                # print ("user", user.playerId, m)
            for user in self.craftUsers:
                x = user.posx
                y = user.posy
                key = str(user.posx) + "_" + str(user.posy)
                if key in self.craftMap:
                    c = self.craftMap[key]
                    del self.craftMap[key]
                    m = min(math.ceil(self.map[y][x] / c), 50)
                    self.map[y][x] -= m * c
                    if self.map[y][x] < 0:
                        self.map[y][x] = 0
                        self.energyOnMap[y][x] = ObstacleInfo.types[0]
                    for g in self.stepState.golds:
                        if g.posx == x and g.posy == y:
                            g.amount = self.map[y][x]
                            if g.amount == 0:
                                self.stepState.golds.remove(g)
                                self.add_changed_obstacle(x, y, 0, ObstacleInfo.types[0])
                                if len(self.stepState.golds) == 0:
                                    for player in self.stepState.players:
                                        player.status = PlayerInfo.STATUS_STOP_EMPTY_GOLD
                            break;
            self.craftMap = {}

    def invalidAction(self, user):
        user.status = PlayerInfo.STATUS_ELIMINATED_INVALID_ACTION
        user.lastAction = 6 #eliminated

    def go_to_pos(self, user):  # player move to cell(x,y)
        if self.map[user.posy][user.posx] == -1:
            user.energy -= randrange(16) + 5
        elif self.map[user.posy][user.posx] == 0:
            user.energy += self.energyOnMap[user.posy][user.posx]
        elif self.map[user.posy][user.posx] == -2:
            user.energy += self.energyOnMap[user.posy][user.posx]
            self.add_changed_obstacle(user.posx, user.posy, 0, ObstacleInfo.types[0])
        elif self.map[user.posy][user.posx] == -3:
            user.energy += self.energyOnMap[user.posy][user.posx]
            self.add_changed_obstacle(user.posx, user.posy, 3,
                                      self.bog_energy_chain[self.energyOnMap[user.posy][user.posx]])
        else:
            user.energy -= 4
        if user.energy <= 0:
            user.status = PlayerInfo.STATUS_ELIMINATED_OUT_OF_ENERGY
            user.lastAction = 6 #eliminated

    def add_changed_obstacle(self, x, y, t, v):
        added = False
        for o in self.stepState.changedObstacles:
            if o["posx"] == x and o["posy"] == y:
                added = True
                break
        if added == False:
            o = {}
            o["posx"] = x
            o["posy"] = y
            o["type"] = t
            o["value"] = v
            self.stepState.changedObstacles.append(o)

    def close(self):
        print("Close socket.")

# DEFAULT BOTS 

## Bot1

In [ ]:
#Bots :bot1
class Bot1:
    ACTION_GO_LEFT = 0
    ACTION_GO_RIGHT = 1
    ACTION_GO_UP = 2
    ACTION_GO_DOWN = 3
    ACTION_FREE = 4
    ACTION_CRAFT = 5

    def __init__(self, id):
        self.state = State()
        self.info = PlayerInfo(id)

    def next_action(self):
        if self.state.mapInfo.gold_amount(self.info.posx, self.info.posy) > 0:
            if self.info.energy >= 6:
                return self.ACTION_CRAFT
            else:
                return self.ACTION_FREE
        if self.info.energy < 5:
            return self.ACTION_FREE
        else:
            action = self.ACTION_GO_UP
            if self.info.posy % 2 == 0:
                if self.info.posx < self.state.mapInfo.max_x:
                    action = self.ACTION_GO_RIGHT
            else:
                if self.info.posx > 0:
                    action = self.ACTION_GO_LEFT
                else:
                    action = self.ACTION_GO_DOWN
            return action

    def new_game(self, data):
        try:
            self.state.init_state(data)
        except Exception as e:
            import traceback
            traceback.print_exc()

    def new_state(self, data):
        # action = self.next_action();
        # self.socket.send(action)
        try:
            self.state.update_state(data)
        except Exception as e:
            import traceback
            traceback.print_exc()


## Bot2

In [ ]:
#Bots :bot2
class Bot2:
    ACTION_GO_LEFT = 0
    ACTION_GO_RIGHT = 1
    ACTION_GO_UP = 2
    ACTION_GO_DOWN = 3
    ACTION_FREE = 4
    ACTION_CRAFT = 5

    def __init__(self, id):
        self.state = State()
        self.info = PlayerInfo(id)

    def next_action(self):
        if self.state.mapInfo.gold_amount(self.info.posx, self.info.posy) > 0:
            if self.info.energy >= 6:
                return self.ACTION_CRAFT
            else:
                return self.ACTION_FREE
        if self.info.energy < 5:
            return self.ACTION_FREE
        else:
            action = np.random.randint(0, 4)            
            return action

    def new_game(self, data):
        try:
            self.state.init_state(data)
        except Exception as e:
            import traceback
            traceback.print_exc()

    def new_state(self, data):
        # action = self.next_action();
        # self.socket.send(action)
        try:
            self.state.update_state(data)
        except Exception as e:
            import traceback
            traceback.print_exc()

## Bot3

In [ ]:
#Bots :bot3
class Bot3:
    ACTION_GO_LEFT = 0
    ACTION_GO_RIGHT = 1
    ACTION_GO_UP = 2
    ACTION_GO_DOWN = 3
    ACTION_FREE = 4
    ACTION_CRAFT = 5

    def __init__(self, id):
        self.state = State()
        self.info = PlayerInfo(id)

    def next_action(self):
        if self.state.mapInfo.gold_amount(self.info.posx, self.info.posy) > 0:
            if self.info.energy >= 6:
                return self.ACTION_CRAFT
            else:
                return self.ACTION_FREE
        if self.info.energy < 5:
            return self.ACTION_FREE
        else:
            action = self.ACTION_GO_LEFT
            if self.info.posx % 2 == 0:
                if self.info.posy < self.state.mapInfo.max_y:
                    action = self.ACTION_GO_DOWN
            else:
                if self.info.posy > 0:
                    action = self.ACTION_GO_UP
                else:
                    action = self.ACTION_GO_RIGHT            
            return action

    def new_game(self, data):
        try:
            self.state.init_state(data)
        except Exception as e:
            import traceback
            traceback.print_exc()

    def new_state(self, data):
        # action = self.next_action();
        # self.socket.send(action)
        try:
            self.state.update_state(data)
        except Exception as e:
            import traceback
            traceback.print_exc()

# MinerState

In [ ]:
#MinerState.py
def str_2_json(str):
    return json.loads(str, encoding="utf-8")


class MapInfo:
    def __init__(self):
        self.max_x = 0 #Width of the map
        self.max_y = 0 #Height of the map
        self.golds = [] #List of the golds in the map
        self.obstacles = []
        self.numberOfPlayers = 0
        self.maxStep = 0 #The maximum number of step is set for this map

    def init_map(self, gameInfo):
        #Initialize the map at the begining of each episode
        self.max_x = gameInfo["width"] - 1
        self.max_y = gameInfo["height"] - 1
        self.golds = gameInfo["golds"]
        self.obstacles = gameInfo["obstacles"]
        self.maxStep = gameInfo["steps"]
        self.numberOfPlayers = gameInfo["numberOfPlayers"]

    def update(self, golds, changedObstacles):
        #Update the map after every step
        self.golds = golds
        for cob in changedObstacles:
            newOb = True
            for ob in self.obstacles:
                if cob["posx"] == ob["posx"] and cob["posy"] == ob["posy"]:
                    newOb = False
                    #print("cell(", cob["posx"], ",", cob["posy"], ") change type from: ", ob["type"], " -> ",
                    #      cob["type"], " / value: ", ob["value"], " -> ", cob["value"])
                    ob["type"] = cob["type"]
                    ob["value"] = cob["value"]
                    break
            if newOb:
                self.obstacles.append(cob)
                #print("new obstacle: ", cob["posx"], ",", cob["posy"], ", type = ", cob["type"], ", value = ",
                #      cob["value"])

    def get_min_x(self):
        return min([cell["posx"] for cell in self.golds])

    def get_max_x(self):
        return max([cell["posx"] for cell in self.golds])

    def get_min_y(self):
        return min([cell["posy"] for cell in self.golds])

    def get_max_y(self):
        return max([cell["posy"] for cell in self.golds])

    def is_row_has_gold(self, y):
        return y in [cell["posy"] for cell in self.golds]

    def is_column_has_gold(self, x):
        return x in [cell["posx"] for cell in self.golds]

    def gold_amount(self, x, y): #Get the amount of golds at cell (x,y)
        for cell in self.golds:
            if x == cell["posx"] and y == cell["posy"]:
                return cell["amount"]
        return 0 

    def get_obstacle(self, x, y):  # Get the kind of the obstacle at cell(x,y)
        for cell in self.obstacles:
            if x == cell["posx"] and y == cell["posy"]:
                return cell["type"]
        return -1  # No obstacle at the cell (x,y)


class State:
    STATUS_PLAYING = 0
    STATUS_ELIMINATED_WENT_OUT_MAP = 1
    STATUS_ELIMINATED_OUT_OF_ENERGY = 2
    STATUS_ELIMINATED_INVALID_ACTION = 3
    STATUS_STOP_EMPTY_GOLD = 4
    STATUS_STOP_END_STEP = 5

    def __init__(self):
        self.end = False
        self.score = 0
        self.lastAction = None
        self.id = 0
        self.x = 0
        self.y = 0
        self.energy = 0
        self.mapInfo = MapInfo()
        self.players = []
        self.stepCount = 0
        self.status = State.STATUS_PLAYING

    def init_state(self, data): #parse data from server into object
        game_info = str_2_json(data)
        self.end = False
        self.score = 0
        self.lastAction = None
        self.id = game_info["playerId"]
        self.x = game_info["posx"]
        self.y = game_info["posy"]
        self.energy = game_info["energy"]
        self.mapInfo.init_map(game_info["gameinfo"])
        self.stepCount = 0
        self.status = State.STATUS_PLAYING
        self.players = [{"playerId": 2, "posx": self.x, "posy": self.y},
                        {"playerId": 3, "posx": self.x, "posy": self.y},
                        {"playerId": 4, "posx": self.x, "posy": self.y}]

    def update_state(self, data):
        new_state = str_2_json(data)
        for player in new_state["players"]:
            if player["playerId"] == self.id:
                self.x = player["posx"]
                self.y = player["posy"]
                self.energy = player["energy"]
                self.score = player["score"]
                self.lastAction = player["lastAction"]
                self.status = player["status"]

        self.mapInfo.update(new_state["golds"], new_state["changedObstacles"])
        self.players = new_state["players"]
        for i in range(len(self.players), 4, 1):
            self.players.append({"playerId": i, "posx": self.x, "posy": self.y})
        self.stepCount = self.stepCount + 1

# Miner Environment

In [ ]:
#MinerEnv.py
TreeID = 1
TrapID = 2
SwampID = 3
class MinerEnv:
    def __init__(self):
        self.socket = GameSocket()
        self.state = State()
        
        self.score_pre = self.state.score#Storing the last score for designing the reward function

    def start(self): #connect to server
        self.socket.connect()

    def end(self): #disconnect server
        self.socket.close()

    def send_map_info(self, request):#tell server which map to run
        self.socket.send(request)

    def reset(self): #start new game
        try:
            message = self.socket.receive() #receive game info from server
            self.state.init_state(message) #init state
        except Exception as e:
            import traceback
            traceback.print_exc()

    def step(self, action): #step process
        self.socket.send(action) #send action to server
        try:
            message = self.socket.receive() #receive new state from server
            self.state.update_state(message) #update to local state
        except Exception as e:
            import traceback
            traceback.print_exc()

    # Functions are customized by client
    def get_state(self):
        # Building the map
        view = np.zeros([self.state.mapInfo.max_x + 1, self.state.mapInfo.max_y + 1], dtype=int)
        for i in range(self.state.mapInfo.max_x + 1):
            for j in range(self.state.mapInfo.max_y + 1):
                if self.state.mapInfo.get_obstacle(i, j) == TreeID:  # Tree
                    view[i, j] = -TreeID
                if self.state.mapInfo.get_obstacle(i, j) == TrapID:  # Trap
                    view[i, j] = -TrapID
                if self.state.mapInfo.get_obstacle(i, j) == SwampID: # Swamp
                    view[i, j] = -SwampID
                if self.state.mapInfo.gold_amount(i, j) > 0:
                    view[i, j] = self.state.mapInfo.gold_amount(i, j)

        DQNState = view.flatten().tolist() #Flattening the map matrix to a vector
        
        # Add position and energy of agent to the DQNState
        DQNState.append(self.state.x)
        DQNState.append(self.state.y)
        DQNState.append(self.state.energy)
        #Add position of bots 
        for player in self.state.players:
            if player["playerId"] != self.state.id:
                DQNState.append(player["posx"])
                DQNState.append(player["posy"])
                
        #Convert the DQNState from list to array for training
        DQNState = np.array(DQNState)

        return DQNState

    def get_reward(self):
        # Calculate reward
        reward = 0
        score_action = self.state.score - self.score_pre
        self.score_pre = self.state.score
        if score_action > 0:
            #If the DQN agent crafts golds, then it should obtain a positive reward (equal score_action)
            reward += score_action
            
        #If the DQN agent crashs into obstacels (Tree, Trap, Swamp), then it should be punished by a negative reward
        if self.state.mapInfo.get_obstacle(self.state.x, self.state.y) == TreeID:  # Tree
            reward -= TreeID
        if self.state.mapInfo.get_obstacle(self.state.x, self.state.y) == TrapID:  # Trap
            reward -= TrapID
        if self.state.mapInfo.get_obstacle(self.state.x, self.state.y) == SwampID:  # Swamp
            reward -= SwampID

        # If out of the map, then the DQN agent should be punished by a larger nagative reward.
        if self.state.status == State.STATUS_ELIMINATED_WENT_OUT_MAP:
            reward += -10
            
        #Run out of energy, then the DQN agent should be punished by a larger nagative reward.
        if self.state.status == State.STATUS_ELIMINATED_OUT_OF_ENERGY:
            reward += -10
        # print ("reward",reward)
        return reward

    def check_terminate(self):
        #Checking the status of the game
        #it indicates the game ends or is playing
        return self.state.status != State.STATUS_PLAYING

# MAPS

In [ ]:
#Creating Maps
#This function is used to create 05 maps instead of loading them from Maps folder in the local
def CreateMaps():
      map1 = [
        [0,  0,  -2,  100,  0,  0,  -1,  -1,  -3,  0,  0,  0,  -1,  -1,  0,  0,  -3,  0,  -1,  -1,0],
        [-1,-1,  -2,  0, 0,  0,  -3,  -1,  0,  -2,  0,  0,  0,  -1,  0,  -1,  0,  -2,  -1,  0,0],
        [0,  0,  -1,  0,  0,  0,  0,  -1,  -1,  -1,  0, 0,  100,  0,  0,  0,  0,  50,  -2,  0,0],
        [0,  0,  0,  0,  -2,  0,  0,  0,  0,  0,  0,  0,  -1,  50, -2,  0,  0,  -1,  -1,  0,0],
        [-2, 0,  200,  -2,  -2,  300,  0, 0,  -2,  -2,  0,  0,  -3,  0,  -1,  0,  0,  -3,  -1,  0,0],
        [0,  -1,  0,  0,  0,  0,  0,  -3,  0,  0,  -1,  -1,  0,  0,  0,  0,  0,  0,  -2,  0,0],
        [0,  -1,  -1,  0,  0,  -1,  -1,  0,  0,  700,  -1,  0,  0,  0,  -2,  -1,  -1,  0,  0, 0,100],
        [0,  0, 0, 500,  0,  0,  -1,  0,  -2,  -2,  -1,  -1,  0,  0,  -2,  0,  -3,  0,  0,  -1,0],
        [-1,  -1, 0,-2 ,  0,  -1,  -2,  0,  400,  -2,  -1,  -1,  500,  0,  -2,  0,  -3,  100,  0, 0,0]
      ]
      map2 = [
        [0,  0,  -2,  0,  0,  0,  -1,  -1,  -3,  0,  0,  0,  -1,  -1,  0,  0,  -3,  0,  -1,  -1,0],
        [-1,-1,  -2,  100, 0,  0,  -3,  -1,  0,  -2,  100,  0,  0,  -1,  0,  -1,  0,  -2,  -1,  0,0],
        [0,  0,  -1,  0,  0,  0,  0,  -1,  -1,  -1,  0, 0,  0,  0,  0,  0,  50,  0,  -2,  0,0],
        [0,  200,  0,  0,  -2,  0,  0,  0,  0,  0,  0,  0,  -1,  50, -2,  0,  0,  -1,  -1,  0,0],
        [-2, 0,  0,  -2,  -2,  0,  0, 0,  -2,  -2,  0,  0,  -3,  0,  -1,  0,  0,  -3,  -1,  0,0],
        [0,  -1,  0,  0,  300,  0,  0,  -3,  0,  0,  -1,  -1,  0,  0,  0,  0,  0,  0,  -2,  0,0],
        [500,  -1,  -1,  0,  0,  -1,  -1,  0,  700,  0,  -1,  0,  0,  0,  -2,  -1,  -1,  0,  0, 0,0],
        [0,  0, 0, 0,  0,  0,  -1,  0,  -2,  -2,  -1,  -1,  0,  0,  -2,  0,  -3,  100,  0,  -1,0],
        [-1,  -1, 0,-2 ,  0,  -1,  -2,  400,  0,  -2,  -1,  -1,  0,  500,  -2,  0,  -3,  0,  0, 100,0]
      ]
      map3= [
        [0,  0,  -2,  0,  0,  0,  -1,  -1,  -3,  0,  100,  0,  -1,  -1,  0,  0,  -3,  0,  -1,  -1,0],
        [-1,-1,  -2,  0, 0,  0,  -3,  -1,  0,  -2,  0,  0,  0,  -1,  0,  -1,  0,  -2,  -1,  0,0 ],
        [0,  0,  -1,  0,  0,  0,  100,  -1,  -1,  -1,  0, 0,  50,  0,  0,  0,  50,  0,  -2,  0,0],
        [0,  200,  0,  0,  -2,  0,  0,  0,  0,  0,  0,  0,  -1,  0, -2,  0,  0,  -1,  -1,  0,0],
        [-2, 0,  0,  -2,  -2,  0,  0, 0,  -2,  -2,  0,  0,  -3,  0,  -1,  0,  0,  -3,  -1,  0,0],
        [0,  -1,  0, 300,  0,  0,  0,  -3,  0,  0,  -1,  -1,  0,  0,  0,  0,  0,  0,  -2,  0,0],
        [0,  -1,  -1,  0,  0,  -1,  -1,  700,  0,  0,  -1,  0,  0,  0,  -2,  -1,  -1,  0,  0, 0,0],
        [0,  0, 0, 0,  0,  500,  -1,  0,  -2,  -2,  -1,  -1,  0,  0,  -2,  0,  -3,  0,  700,  -1,0],
        [-1,  -1, 0,-2 ,  0,  -1,  -2,  400,  0,  -2,  -1,  -1,  0,  500,  -2,  0,  -3,  0,  0, 100,0]
      ]
      map4=[
        [0,  0,  -2,  0,  0,  0,  -1,  -1,  -3,  0,  0,  0,  -1,  -1,  0,  0,  -3,  0,  -1,  -1,0],
        [-1,-1,  -2,  0, 0,  0,  -3,  -1,  0,  -2,  0,  0,  100,  -1,  0,  -1,  0,  -2,  -1,  0,0],
        [0,  0,  -1,  0,  100,  0,  0,  -1,  -1,  -1,  0, 0,  0,  0,  50,  0,  50,  0,  -2,  0,0],
        [0,  200,  0,  0,  -2,  0,  0,  0,  0,  0,  0,  0,  -1,  0, -2,  0,  0,  -1,  -1,  0,0],
        [-2, 0,  0,  -2,  -2,  0,  0, 0,  -2,  -2,  0,  0,  -3,  0,  -1,  0,  0,  -3,  -1,  0,0],
        [0,  -1,  0,  0,  0,  0,  300,  -3,  0,  700,  -1,  -1,  0,  0,  0,  0,  0,  0,  -2,  0,0],
        [0,  -1,  -1,  0,  0,  -1,  -1,  0,  0,  0,  -1,  0,  0,  0,  -2,  -1,  -1,  0,  0, 100,0],
        [500,  0, 0, 0,  0,  0,  -1,  0,  -2,  -2,  -1,  -1,  0,  0,  -2,  0,  -3,  0,  0,  -1,0],
        [-1,  -1, 0,-2 ,  0,  -1,  -2,  400,  0,  -2,  -1,  -1,  0,  500,  -2,  0,  -3,  0,  0, 100,0]

      ]
      map5=[
        [0,  0,  -2,  0,  100,  0,  -1,  -1,  -3,  0,  0,  0,  -1,  -1,  0,  0,  -3,  0,  -1,  -1,0],
        [-1,-1,  -2,  0, 0,  0,  -3,  -1,  0,  -2,  100,  0,  0,  -1,  0,  -1,  0,  -2,  -1,  0,0],
        [0,  0,  -1,  0,  0,  0,  0,  -1,  -1,  -1,  0, 0,  0,  0,  50,  0,  0,  0,  -2,  0,0],
        [0,  200,  0,  0,  -2,  0,  0,  0,  0,  0,  0,  0,  -1,  0, -2,  0,  50,  -1,  -1,  0,0],
        [-2, 0,  0,  -2,  -2,  0,  0, 0,  -2,  -2,  0,  0,  -3,  0,  -1,  0,  0,  -3,  -1,  0,0],
        [0,  -1,  0,  0,  300,  0,  0,  -3,  0,  0,  -1,  -1,  0,  0,  0,  0,  0,  0,  -2,  0,0],
        [500,  -1,  -1,  0,  0,  -1,  -1,  0,  0,  700,  -1,  0,  0,  0,  -2,  -1,  -1,  0,  0, 100,0],
        [0,  0, 0, 0,  0,  0,  -1,  0,  -2,  -2,  -1,  -1,  0,  0,  -2,  0,  -3,  0,  0,  -1,0],
        [-1,  -1, 0,-2 ,  0,  -1,  -2,  400,  0,  -2,  -1,  -1,  0,  500,  -2,  0,  -3,  0,  0, 100,0]
      ]
      Maps = (map1,map2,map3,map4,map5)
      return Maps   


# Replay Buffer

In [ ]:
class ReplayBuffer: 
    def __init__(
        self,
        max_size, # max size of buffer  
        input_shape, 
        n_actions, # The number of action 
        discrete = False
    ):
        self.mem_size = max_size
        self.mem_counter = 0 # Contain index of last element 
        self.discrete = discrete 
        self.state_memory = np.zeros((self.mem_size, input_shape))
        self.new_state_memory = np.zeros((self.mem_size, input_shape))

        dtype = np.int8 if self.discrete else np.float32 # type setting for memory saving
        self.action_memory = np.zeros((self.mem_size, n_actions), dtype = dtype) 
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype = np.float32)

    def store_transition(self, state, action, reward, state_next, done): 
        index = self.mem_counter % self.mem_size # TODO: not clear this line
        self.new_state_memory[index] = state_next
        if self.discrete: 
            actions = np.zeros(self.action_memory.shape[1])
            actions[action] = 1.0 
            self.action_memory[index] = actions  
        else: 
            self.action_memory[index] = action 
        self.reward_memory[index] = reward 
        self.terminal_memory[index] = 1 - int(done) # flag 
        self.mem_counter = self.mem_counter + 1

    def sample_buffer(self, batch_size): 
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem, batch_size)
        states = self.state_memory[batch]
        new_states = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        terminal = self.terminal_memory[batch]
        rewards = self.reward_memory[batch]

        return states, actions, rewards, new_states, terminal

# Build DQN model

In [ ]:
def build_dqn(learning_rate, n_actions, input_dims, fc1_dims, fc2_dims): 
    u''' 
        fc1_dims : fully connected layers dimension
        fc2_dims : fully connected layers dimension
    '''
    model = Sequential([
        Dense(fc1_dims, input_dim = input_dims), 
        Activation('relu'), 
        Dense(fc2_dims), 
        Activation('relu'), 
        Dense(n_actions), 
    ])
    model.compile(optimizer=Adam(lr=learning_rate), loss='mse')
    return model
    # model = Sequential([
    #     Dense(fc1_dims, input_shape(input_dims,)), 
    #     Activation('relu'), 
    #     Dense(fc2_dims), 
    #     Activation('relu'), 
    #     Dense(n_actions), 
    # ])
    # model.compile(optimizer=Adam(lr=learning_rate), loss='mse')
    # return model

# Double Deep Q Network off-policy

In [ ]:
# Double Deep Q Network off-policy 
class DDQNAgent: 
    def __init__(
        self, 
        input_dims, # input dimension   
        n_actions, # the total numbere of actions 
        batch_size, # batch size which taken from memory for experience replay  
        alpha = 0.00025, # learning rate
        gamma = 0.99, # discount factor
        epsilon = 1, # exploration factor for epsilon greedy policy  
        epsilon_decay = 0.9999, 
        epsilon_min = 0.01, 
        fname = 'ddqn_model.h5', 
        mem_size = 100000, 
        replace_target = 300, # replace target network with online network 
    ): 
        # print("Debug DDQNAgent, constructor called")
        self.n_actions = n_actions
        self.action_space = [i for i in range(self.n_actions)]
        self.batch_size = batch_size
        self.replace_target = replace_target
        self.model = fname 
        self.memory = ReplayBuffer(mem_size, input_dims, n_actions, discrete=True)

        # Hyperparameters 
        self.gamma = gamma 
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        # Creating network for evaluation and target
        self.q_eval = build_dqn(alpha, n_actions, input_dims, 300, 300) 
        self.q_target = build_dqn(alpha, n_actions, input_dims, 300, 300) 

    def model_info(self): 
      self.q_eval.summary()
      self.q_target.summary()

    def remember(self, state, action, reward, new_state, done): 
        self.memory.store_transition(state, action, reward, new_state, done) 

    # Action for each state with epsilon greedy policy
    def choose_act(self,state): 
      state = state.reshape(1,len(state))
      a_max = np.argmax(self.q_eval.predict(state))
      if random() < self.epsilon: 
        a_chosen = randrange(self.n_actions)
      else: 
        a_chosen = a_max 
      return a_chosen


    # def experience_replay(self): 
      




    # TODO: NEED FURTHER UNDERSTANDING
    def learn(self): 
        if self.memory.mem_counter > self.batch_size: 
            state, action, reward, new_state, done = self.memory.sample_buffer(
                                                                self.batch_size) 
            action_values = np.array(self.action_space, dtype = np.int8)
            action_indices = np.dot(action, action_values)

            q_next = self.q_target.predict(new_state)
            q_eval = self.q_eval.predict(new_state)

            q_pred = self.q_eval.predict(state) 

            max_actions = np.argmax(q_eval, axis = 1)

            q_target = q_pred 
            batch_index = np.arange(self.batch_size, dtype = np.int32) 

            q_target[batch_index, action_indices] = reward + self.gamma*q_next[batch_index, max_actions.astype(int)]*done

            _ = self.q_eval.fit(state, q_target, verbose = 0)
            
            # # Decay epsilon after each iteration 
            # self.epsilon = self.epsilon*self.epsilon_decay if self.epsilon > self.epsilon_min else self.epsilon_min

            if self.memory.mem_counter % self.replace_target == 0: 
                self.update_network_parameters()

    def update_epsilon(self): 
      self.epsilon = self.epsilon*self.epsilon_decay
      self.epsilon = max(self.epsilon_min,self.epsilon)


    # TODO: FIXED  
    # def update_network_parameters(self): 
    #     self.q_target.model.set_weights(self.q_eval.model.get_weights()) # this line, problem with model.
    def update_network_parameters(self): 
        self.q_target.set_weights(self.q_eval.get_weights())
  
    # Based on DQNModel from RLComp2020 source code
    def save_model(self, path, model_name): 
      model_json = self.q_eval.to_json()
      with open(path + model_name + ".json", "w") as json_file: 
        json_file.write(model_json)
        # serialize weights to HDF5
        self.q_eval.save_weights(path + model_name + ".h5")
        print("Saved model to disk")
    
    # # DID NOT TEST YET 
    # def load_model(self): 
    #     self.q_eval.load_model(self.model_file)
    #     if self.epsilon <= self.epsilon_min: 
    #         self.update_network_parameters()

# DDQN with Lunar Lander v2 in Gym from OpenAI

In [ ]:
# MAIN TRAINING ALGORITHM FOR DDQN

# import gym
# # from gym import wrappers 
# import numpy as np

# env = gym.make('LunarLander-v2')

# # ddqn_agent = DDQNAgent(alpha = 0.0005, gamma = 0.99, epsilon = 1.0, 
# #                         input_dims=8, batch_size = 64, n_actions = 4) 
# ddqn_agent = DDQNAgent(input_dims = 8, n_actions = 4, batch_size = 64)
# n_games = 500 
# ddqn_scores = []
# eps_history = []

# # Print model info
# ddqn_agent.model_info()

# for i in range(n_games): 
#   done = False 
#   score = 0 
#   observation = env.reset()
#   while not done: 
#       action = ddqn_agent.choose_act(observation)
#       observation_, reward, done, info = env.step(action) 
#       score += reward 
#       ddqn_agent.remember(observation, action, reward, observation_, done)
#       observation = observation_
#       ddqn_agent.learn()
#   eps_history.append(ddqn_agent.epsilon)
#   ddqn_scores.append(score)

#   avg_score = np.mean(ddqn_scores[max(0, i-100):(i+1)])

#   # print('episode ',i,'score %.2f ' % score, 'average score %.2f' % avg_score)
#   print('----------------------------------------')
#   print('episode ', i)
#   print('score %.2f' %score)
#   print('average score %.2f' %avg_score)
#   print('----------------------------------------')

#   # DEBUG 
#   if i == 5: 
#     ddqn_agent.save_model("/", "DDQNModel_" + "ep_" + str(i))


#   # if i % 100 == 0 and i > 0: 
#   #     ddqn_agent.save_model()


# Main DDQN Training Algorithm

## Sample main traninig algorithm from RLComp sample source code

In [ ]:
# # Initialize network and memory
# # DQNAgent = DQN(INPUTNUM,ACTIONNUM)
# # memory = Memory(MEMORY_SIZE)

# # Initialize environment
# Maps = CreateMaps()#Creating 05 maps
# minerEnv = MinerEnv()#Creating a communication environment between the DQN model and the game environment
# minerEnv.start() #Connect to the game

# train = False #The variable is used to indicate that the replay starts, and the epsilon starts decrease.
# #Training Process
# #the main part of the deep-q learning agorithm 
# for episode_i in range(0,N_EPISODE):
#     try:
#         # Choosing a map in the list
#         mapID = np.random.randint(0, 5) #Choosing a map ID from 5 maps in Maps folder randomly
#         posID_x = np.random.randint(MAP_MAX_X) #Choosing a initial position of the DQN agent on X-axes randomly
#         posID_y = np.random.randint(MAP_MAX_Y) #Choosing a initial position of the DQN agent on Y-axes randomly
#         #Creating a request for initializing a map, initial position, the initial energy, and the maximum number of steps of the DQN agent
#         request = ("map" + str(mapID) + "," + str(posID_x) + "," + str(posID_y) + ",50,100") 
#         #Send the request to the game environment
#         minerEnv.send_map_info(request)

#         # Getting the initial state
#         minerEnv.reset() #Initialize the game environment
#         s = minerEnv.get_state()#Get the state after reseting. 
#                                 #This function (get_state()) is an example of creating a state for the DQN model 
#         total_reward = 0 #The amount of rewards for the entire episode
#         terminate = False #The variable indicates that the episode ends
#         maxStep = minerEnv.state.mapInfo.maxStep #Get the maximum number of steps for each episode in training
#         #Start an episde for training
#         for step in range(0, maxStep):
#             action = DQNAgent.act(s)  # Getting an action from the DQN model from the state (s)
#             minerEnv.step(str(action))  # Performing the action in order to obtain the new state
#             s_next = minerEnv.get_state()  # Getting a new state
#             reward = minerEnv.get_reward()  # Getting a reward
#             terminate = minerEnv.check_terminate()  # Checking the end status of the episode
             
#             # Add this transition to the memory batch
#             memory.push(s, action, reward, terminate, s_next)

#             # Sample batch memory to train network
#             if (memory.length > INITIAL_REPLAY_SIZE):
#                 #If there are INITIAL_REPLAY_SIZE experiences in the memory batch
#                 #then start replaying
#                 batch = memory.sample(BATCH_SIZE) #Get a BATCH_SIZE experiences for replaying
#                 DQNAgent.replay(batch, BATCH_SIZE)#Do relaying
#                 train = True #Indicate the training starts
#             total_reward = total_reward + reward #Plus the reward to the total rewad of the episode
#             s = s_next #Assign the next state for the next step.
            
#             #Saving data to file
#             '''save_data = np.hstack([episode_i+1,step+1,reward,total_reward,action, DQNAgent.epsilon, terminate]).reshape(1,7)
#             with open(filename, 'a') as f:
#                 pd.DataFrame(save_data).to_csv(f,encoding='utf-8', index=False, header = False)'''

#             if terminate == True:
#                 #If the episode ends, then go to the next episode
#                 break
            
#         # Iteration to save the network architecture and weights
#         if (np.mod(episode_i + 1, SAVE_NETWORK) == 0 and train == True):
#             DQNAgent.target_train()  # Replace the learning weights for target model with soft replacement
#             #Save the DQN model
#             now = datetime.datetime.now() #Get the latest datetime          
#             DQNAgent.save_model( "DQNmodel_" + now.strftime("%Y%m%d-%H%M") + "_ep" + str(episode_i+1))   
        
#         #Print the training information after the episode
#         print('Episode %d ends. Number of steps is: %d. Accumlated Reward = %.2f. Epsilon = %.2f .Termination code: %d' % (episode_i+1, step+1, total_reward, DQNAgent.epsilon, terminate))
#         #Decreasing the epsilon if the replay starts
#         if  train == True:
#             DQNAgent.update_epsilon()
            	
#     except Exception as e:
#         import traceback
#         traceback.print_exc()                
#         #print("Finished.")
#         break

# DDQN Gold Miner parameter settings

In [ ]:
# Parameters for training a DQN model
N_EPISODE = 100000 #The number of episodes for training
MAX_STEP = 1000   #The number of steps for each episode
BATCH_SIZE = 64   #The number of experiences for each replay 
MEMORY_SIZE = 1000000 #The size of the batch for storing experiences
SAVE_NETWORK = 100  # After this number of episodes, the DQN model is saved for testing later. 
INITIAL_REPLAY_SIZE = 1000 #The number of experiences are stored in the memory batch before starting replaying
INPUTNUM = 198 #The number of input values for the DQN model
ACTIONNUM = 6  #The number of actions output from the DQN model
MAP_MAX_X = 21 #Width of the Map
MAP_MAX_Y = 9  #Height of the Map

In [ ]:
# Initialize environment
Maps = CreateMaps()#Creating 05 maps
minerEnv = MinerEnv()#Creating a communication environment between the DQN model and the game environment
minerEnv.start() #Connect to the game

# train = False #The variable is used to indicate that the replay starts, and the epsilon starts decrease.

Connected to server.
Found: map0
Found: map1
Found: map2
Found: map3
Found: map4


In [ ]:
ddqn_agent = DDQNAgent(input_dims = INPUTNUM, n_actions = ACTIONNUM, batch_size = BATCH_SIZE)
n_games = 500 
ddqn_scores = []
eps_history = []

In [ ]:
ddqn_agent.model_info()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               59700     
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
activation_1 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1806      
Total params: 151,806
Trainable params: 151,806
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                

In [ ]:
# # Debug purpose 
# ddqn_agent.return_eps()
# print(random())

# DDQN Main Training Algorithm for Gold Miner Environment

In [ ]:
# for episode_i in range(0,N_EPISODE):
#   try: 
#       # Choosing a map in the list
#       mapID = np.random.randint(0, 5) #Choosing a map ID from 5 maps in Maps folder randomly
#       posID_x = np.random.randint(MAP_MAX_X) #Choosing a initial position of the DQN agent on X-axes randomly
#       posID_y = np.random.randint(MAP_MAX_Y) #Choosing a initial position of the DQN agent on Y-axes randomly
      
#       #Creating a request for initializing a map, initial position, the initial energy, and the maximum number of steps of the DQN agent
#       request = ("map" + str(mapID) + "," + str(posID_x) + "," + str(posID_y) + ",50,100") 
#       #Send the request to the game environment (GAME_SOCKET_DUMMY.py)
#       minerEnv.send_map_info(request)

#       # Getting the initial state
#       minerEnv.reset() #Initialize the game environment
#       s = minerEnv.get_state()#Get the state after reseting. 
#                               #This function (get_state()) is an example of creating a state for the DQN model 
#       maxStep = minerEnv.state.mapInfo.maxStep # Get the maximum number of steps for each episode in training
#       done = False # Flag indicates that the episode ends
#       score = 0 # Score for entire episode

#       # for debug purpose
#       list_action_per_eps = []
#       while not done: 
#         action = ddqn_agent.choose_act(s) # Getting action from the model
#         minerEnv.step(str(action)) # Performing the action in order to obtain new state
#         done = minerEnv.check_terminate() # Checking the end status of the episode
#         s_next = minerEnv.get_state() # Getting a new state
#         reward = minerEnv.get_reward() # Getting a reward
#         # list_action_per_eps.append(action)
#         # Store current state and perform learning for current state
#         ddqn_agent.remember(s,action, reward, s_next,done) # Push current state to memory for experience replay
#         ddqn_agent.learn() # Performing learn and push current state to the memory
        
#         score += reward # Total reward of an episode
#         s = s_next # Assign the next state for the next step
#       eps_history.append(ddqn_agent.epsilon)
#       ddqn_scores.append(score)

#       avg_score = np.mean(ddqn_scores[max(0, episode_i-100):(episode_i+1)])

#       # print('episode ',i,'score %.2f ' % score, 'average score %.2f' % avg_score)
#       print('----------------------------------------')
#       print('episode ', episode_i)
#       print('score %.2f' %score)
#       print('epsilon  %.2f' %ddqn_agent.epsilon)
#       print('average score %.2f' %avg_score)
#       print('list of action: ', list_action_per_eps)
#       print('----------------------------------------')

#       # Save DDQN model for every 100 iteration
#       if episode_i % SAVE_NETWORK == 0 and episode_i > 0: 
#         ddqn_agent.save_model("/", "DDQNModel_" + "ep_" + str(episode_i))    

#     # # Save DDQN model for every 100 iteration
#     # if episode_i % 100 == 0: 
#     #   ddqn_agent.save_model("/", "DDQNModel_episode_" + str(i))

#   except Exception as e: 
#     import traceback
#     traceback.print_exc()
#     break

Reset game:  ['map3', '12', '4', '50', '100']
numberOfPlayers:  4


Traceback (most recent call last):
  File "<ipython-input-34-d34412b08a89>", line 24, in <module>
    action = ddqn_agent.choose_act(s) # Getting action from the model
NameError: name 'ddqn_agent' is not defined


# [not done] DDQN main training with maxStep per episode implementation

In [ ]:
for episode_i in range(0,N_EPISODE):
  try: 
      # Choosing a map in the list
      mapID = np.random.randint(0, 5) #Choosing a map ID from 5 maps in Maps folder randomly
      posID_x = np.random.randint(MAP_MAX_X) #Choosing a initial position of the DQN agent on X-axes randomly
      posID_y = np.random.randint(MAP_MAX_Y) #Choosing a initial position of the DQN agent on Y-axes randomly
      
      #Creating a request for initializing a map, initial position, the initial energy, and the maximum number of steps of the DQN agent
      request = ("map" + str(mapID) + "," + str(posID_x) + "," + str(posID_y) + ",50,100") 
      #Send the request to the game environment (GAME_SOCKET_DUMMY.py)
      minerEnv.send_map_info(request)

      # Getting the initial state
      minerEnv.reset() #Initialize the game environment
      s = minerEnv.get_state()#Get the state after reseting. 
                              #This function (get_state()) is an example of creating a state for the DQN model 
      maxStep = minerEnv.state.mapInfo.maxStep # Get the maximum number of steps for each episode in training
      done = False # Flag indicates that the episode ends
      score = 0 # Score for entire episode

      for step in range(0,maxStep):
        action = ddqn_agent.choose_act(s) # Getting action from the model
        minerEnv.step(str(action)) # Performing the action in order to obtain new state
        s_next = minerEnv.get_state() # Getting a new state
        reward = minerEnv.get_reward() # Getting a reward
        done = minerEnv.check_terminate() # Checking the end status of the episode
        # list_action_per_eps.append(action) # For debugging purpose
        
        # Store current state and perform learning for current state
        ddqn_agent.remember(s,action, reward, s_next,done) # Push current state to memory for experience replay
        ddqn_agent.learn() # Performing learn and push current state to the memory
        score += reward # Total reward of an episode
        s = s_next # Assign the next state for the next step
        if done == True: 
          break
      # Update epsilon for every episode
      ddqn_agent.update_epsilon()
        
      # For output purpose
      eps_history.append(ddqn_agent.epsilon)
      ddqn_scores.append(score)
      avg_score = np.mean(ddqn_scores[max(0, episode_i-100):(episode_i+1)])

      print('----------------------------------------')
      print('episode ', episode_i)
      print('score %.2f' %score)
      print('epsilon  %.2f' %ddqn_agent.epsilon)
      print('average score %.2f' %avg_score)
      print('----------------------------------------')

      # Save DDQN model for every 100 iteration
      if episode_i % SAVE_NETWORK == 0 and episode_i > 0: 
        ddqn_agent.save_model("/", "DDQNModel_" + "ep_" + str(episode_i))    

  except Exception as e: 
    import traceback
    traceback.print_exc()
    break

Streaming output truncated to the last 5000 lines.
numberOfPlayers:  4
----------------------------------------
episode  30911
score -28.00
epsilon  0.05
average score -25.97
----------------------------------------
Reset game:  ['map1', '1', '2', '50', '100']
numberOfPlayers:  4
----------------------------------------
episode  30912
score 50.00
epsilon  0.05
average score -25.26
----------------------------------------
Reset game:  ['map0', '3', '1', '50', '100']
numberOfPlayers:  4
----------------------------------------
episode  30913
score -11.00
epsilon  0.05
average score -24.77
----------------------------------------
Reset game:  ['map3', '20', '8', '50', '100']
numberOfPlayers:  4
----------------------------------------
episode  30914
score -10.00
epsilon  0.05
average score -24.87
----------------------------------------
Reset game:  ['map0', '15', '8', '50', '100']
numberOfPlayers:  4
----------------------------------------
episode  30915
score -156.00
epsilon  0.05
aver

KeyboardInterrupt: ignored

# Plotting training process

In [ ]:
import pandas as pd

In [ ]:
# eps_history
# ddqn_scores
# avg_score 